# 1. Introduction

This notebook connects to the Fresh Produce Database on SQL, cleans the tables and save the cleaned tables in the database.

!pip install pyodbc 

In [ ]:
#!pip install sqlalchemy
#!conda install pymssql
#apt-get -qq install python-pymssql python3-pymssql

In [1]:
#Importing modules
import pandas as pd
import numpy as np
import pyodbc

## Local Connection SQL

conn = pyodbc.connect('Driver={SQL Server};'
                      'Server='
                      'Database='
                      'Trusted_Connection=yes;')

cursor = conn.cursor()

cpt_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.Capetown_Fresh_produce_market',conn)
dbn_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.Durban_Fresh_produce_market',conn)
jhb_comm_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.Joburg_Fresh_produce_commodity_raw',conn)
jhb_con_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.Joburg_Fresh_produce_container_raw',conn)
jhb_comb_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.Joburg_Fresh_produce_product_combination_raw',conn)
woolworths_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.woolworths_Prices',conn)
pnp_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.PickNPay_Prices',conn)
shoprite_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.Shoprite_Prices',conn)
#print(sql_query)
#print(type(sql_query))

## Remote Connection (RDS)

In [1]:
# creating a connection to the remote database (RDS)
cnxn = pyodbc.connect(
    DRIVER='{SQL Server}',
    SERVER='',
    DATABASE='',
    uid='',
    pwd='') 

NameError: name 'pyodbc' is not defined

In [3]:
# For sql queries 
cur = cnxn.cursor()

In [4]:
#Creating the pandas dataframe to store the data imported from RDS instance:
cpt_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.Capetown_Fresh_produce_market',cnxn)
dbn_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.Durban_Fresh_produce_market',cnxn)
jhb_comm_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.Joburg_Fresh_produce_commodity_raw',cnxn)
jhb_con_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.Joburg_Fresh_produce_container_raw',cnxn)
jhb_comb_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.Joburg_Fresh_produce_product_combination_raw',cnxn)
woolworths_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.woolworths_Prices',cnxn)
pnp_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.PickNPay_Prices',cnxn)
shoprite_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.Shoprite_Prices',cnxn)

In [5]:
# Exploring the data
cpt_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4290 entries, 0 to 4289
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   item             4290 non-null   object
 1   description      4290 non-null   object
 2   grade            4290 non-null   object
 3   container        4290 non-null   object
 4   container_count  4290 non-null   int64 
 5   mass             4290 non-null   object
 6   low_price        4290 non-null   object
 7   high_price       4290 non-null   object
 8   avg_price        4290 non-null   object
 9   date_updated     4290 non-null   object
dtypes: int64(1), object(9)
memory usage: 335.3+ KB


In [7]:
#jhb_comm_df.head()
#jhb_con_df.head()
jhb_comb_df.head()

,rowid,date,commodity,container,unit_mass,product_combination,total_value_sold,total_qty_sold,total_kg_sold,average,highest_price,ave_per_kg,highest_price_per_kg
0,13803,31 August 2020,ONIONS,2KG POCKET,2,"SWEET,CL 1,L/M,*,NORTHERN CAPE",R0.00,0,0.0,R0.00,R0.00,R0.00,R0.00
1,13804,31 August 2020,ONIONS,2KG POCKET,2,"WHITE,CL 1,L/M,*,NORTHERN CAPE",R0.00,0,0.0,R0.00,R0.00,R0.00,R0.00
2,13805,31 August 2020,ONIONS,2KG POCKET,2,"WHITE,CL 1,M,*,NORTHERN CAPE",R0.00,0,0.0,R0.00,R0.00,R0.00,R0.00
3,13806,31 August 2020,ONIONS,3KG POCKET,3,"BROWN,CL 1,L/M,*,NORTHERN CAPE","R6,138.00",361,1083.0,R17.00,R18.00,R5.67,R6.00
4,13807,31 August 2020,ONIONS,3KG POCKET,3,"BROWN,CL 1,L/M,*,WESTERN CAPE",R0.00,0,0.0,R0.00,R0.00,R0.00,R0.00


# Creating Functions to clean the data

## The function to clean Rand Value columns  and normalise them

In [20]:
def rand_value(df, column):
    """
    Takes a dataframe and a column with rand value to be cleaned as arguments, 
    returns a dataframe, with two new columns with rand value for one day and
    Month To Date rand value. The initial column that was added as argument is removed from the dataframe.
    """ 
    
    df["MTD_total_value_sold_(R)"]=np.nan
    df["total_value_sold_(R)"]=np.nan
    for i in range(len(df[column])):
        df["MTD_total_value_sold_(R)"][i]=round(float(df[column][i].split("R")[-1].replace(',','')),2)
        df["total_value_sold_(R)"][i]=round(float(df[column][i].split("R")[1].split("M")[0].replace(",","")),2)
    
    df1=df.copy()    
    df1.drop(column,axis=1, inplace=True)
    return df1
    

## Cleaning the Johanneburg market Rand Values 
* Using the function

In [21]:
# cleaning the rand value column of commodity table and storing it in a new dataframe
jhb_comm_df_cleaned=rand_value(jhb_comm_df,"total_value_sold")
# cleaning the rand value column of the container dataframe and storing
jhb_con_df_cleaned=rand_value(jhb_con_df,"value_sold")

C:\Users\27784\anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\27784\anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [22]:
#Visualising the new dataframes
jhb_comm_df_cleaned.head()
jhb_con_df_cleaned.head()

,rowid,date,commodity,container,qty_available,qty_sold,kg_sold,average_price_per_kg,MTD_total_value_sold_(R),total_value_sold_(R)
0,1,20 August 2020,AMADUMBE,20KG POCKET,2,0MTD: 97,"0MTD: 1,940",R0,39870.0,0.0
1,2,20 August 2020,APPLES,10 X 1KG ECONO PACK CARTON,17,1MTD: 547,"10MTD: 5,470",R10,41932.0,100.0
2,3,20 August 2020,APPLES,11KG JUMBLE CARTON,343,"17MTD: 3,170","187MTD: 34,870",R6.36,218914.0,1190.0
3,4,20 August 2020,APPLES,12 X 1KG ECONO PACK CARTON,3233,"432MTD: 17,353","5184MTD: 208,236",R7.86,1454572.0,40738.0
4,5,20 August 2020,APPLES,12.5KG M6 CARTON,12,0MTD: 286,"0MTD: 3,575",R0,16903.0,0.0


## The function to clean quantities and weight columns simultaneuosly

In [23]:
def sold_qty_kg(df,quatity_sold, weight_sold):
    """
    Takes a dataframe and a two column with quantity sold and weight sold to be cleaned as arguments. 
    Returns a dataframe, with four new columns with measurements for one day and month to date measurements,
    of quantity sold and weight sold for one day and Month To Date. The initial columns that were added as argument 
    are removed from the new dataframe that is returned.
    """ 
    
    df["Total_quatity_sold"]=np.nan
    df["MTD_Total_quatity_sold"]=np.nan
    df["Total_kg_sold"]=np.nan
    df["MTD_total_kg_sold"]=np.nan
    for i in range(len(df[quatity_sold])):
        df["Total_quatity_sold"][i]=int(df[quatity_sold][i].split("M")[0].replace(",",""))
        df["MTD_Total_quatity_sold"][i]=int(df[quatity_sold][i].split(":")[-1].replace(',',''))
        df["Total_kg_sold"][i]=float(df[weight_sold][i].split("M")[0].replace(",",""))
        df["MTD_total_kg_sold"][i]=float(df[weight_sold][i].split(":")[-1].replace(',',''))
    df1=df.copy()
    df1.drop([quatity_sold,weight_sold],axis=1, inplace=True)
    return df1

## Cleaning the Joburg Market taables' quantity sold and weight sold column

In [24]:
jhb_comm_df_cleaned= sold_qty_kg(jhb_comm_df_cleaned, "total_qty_sold","total_kg_sold")
jhb_con_df_cleaned= sold_qty_kg(jhb_con_df_cleaned, "qty_sold","kg_sold")

C:\Users\27784\anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\27784\anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
C:\Users\27784\anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
C:\Users\27784\anaconda3\lib

In [25]:
# Looking at the first five rows
#jhb_comm_df_cleaned.head()
jhb_con_df_cleaned.head()

,rowid,date,commodity,container,qty_available,average_price_per_kg,MTD_total_value_sold_(R),total_value_sold_(R),Total_quatity_sold,MTD_Total_quatity_sold,Total_kg_sold,MTD_total_kg_sold
0,1,20 August 2020,AMADUMBE,20KG POCKET,2,R0,39870.0,0.0,0.0,97.0,0.0,1940.0
1,2,20 August 2020,APPLES,10 X 1KG ECONO PACK CARTON,17,R10,41932.0,100.0,1.0,547.0,10.0,5470.0
2,3,20 August 2020,APPLES,11KG JUMBLE CARTON,343,R6.36,218914.0,1190.0,17.0,3170.0,187.0,34870.0
3,4,20 August 2020,APPLES,12 X 1KG ECONO PACK CARTON,3233,R7.86,1454572.0,40738.0,432.0,17353.0,5184.0,208236.0
4,5,20 August 2020,APPLES,12.5KG M6 CARTON,12,R0,16903.0,0.0,0.0,286.0,0.0,3575.0


## The function to change columns to numeric 

In [8]:
def to_numeric(df,column):
    """
    Takes a dataframe and one column that have numbers and a comma as a character.
    Removes the comma and change the column to numeric.
    Returns a dataframe with the column changed to be a numeric column
    """ 
    for i in range(len(df[column])):
        df[column][i]=df[column][i].replace("R","").replace(",","")
    df[column]= pd.to_numeric(df[column])
    return  df

## Changing the columns in the dataframes to be numric

In [9]:
# Cleaning CapeTown Dataframe
cpt_df["date_updated"]=pd.to_datetime(cpt_df["date_updated"])
cpt_df=to_numeric(cpt_df,"mass")
cpt_df=to_numeric(cpt_df, "low_price")
cpt_df=to_numeric(cpt_df,"high_price")
cpt_df=to_numeric(cpt_df,"avg_price")

C:\Users\27784\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [10]:
cpt_df.head()

,item,description,grade,container,container_count,mass,low_price,high_price,avg_price,date_updated
0,APCR,APPLE CRIPPS RED,1M,ECONOPACK (12kg),150,12.0,110.0,110.0,110.00,2020-08-21 11:36:00
1,APER,APPLE EARLY RED ONE,1L,MARK 4 (18.3kg),100,18.3,120.0,120.0,120.00,2020-08-21 11:36:00
2,APER,APPLE EARLY RED ONE,1M,MARK 4 (18.3kg),135,18.3,120.0,140.0,121.23,2020-08-21 11:36:00
3,APER,APPLE EARLY RED ONE,1X,MARK 4 (18.3kg),90,18.3,120.0,120.0,120.00,2020-08-21 11:36:00
4,APGA,APPLE GALA,1L,MARK 4 (18.3kg),110,18.3,160.0,160.0,160.00,2020-08-21 11:36:00


In [26]:
#Cleaning durban dataframe
dbn_df["Date"]=pd.to_datetime(dbn_df["Date"])
dbn_df=to_numeric(dbn_df,"Weight_Kg")
dbn_df=to_numeric(dbn_df,"Low_Price")
dbn_df=to_numeric(dbn_df,"High_Price")
dbn_df=to_numeric(dbn_df,"Average_Price")
dbn_df=to_numeric(dbn_df,"Sales_Total")
dbn_df=to_numeric(dbn_df,"Total_Qty_Sold")
dbn_df=to_numeric(dbn_df,"Total_Kg_Sold")
dbn_df=to_numeric(dbn_df,"Stock_On_Hand")
dbn_df.head()

C:\Users\27784\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Commodities,Weight_Kg,Size_Grade,Container,Province,Low_Price,High_Price,Average_Price,Sales_Total,Total_Qty_Sold,Total_Kg_Sold,Stock_On_Hand,Date
0,NOVAS,10.0,2M,BI100,CAPE,0.0,0.0,0.00,0.0,0,0.0,52,2020-07-15
1,NOVAS,10.0,2M,BI100,EASTERN CAPE,65.0,65.0,65.00,14560.0,224,2240.0,0,2020-07-15
2,NOVAS,15.0,2M,BN150,CAPE,70.0,75.0,70.71,495.0,7,105.0,0,2020-07-15
3,NOVAS,10.0,2S,BI100,CAPE,30.0,30.0,30.00,180.0,6,60.0,70,2020-07-15
4,NOVAS,10.0,2S,BI100,EASTERN CAPE,50.0,50.0,50.00,21700.0,434,4340.0,0,2020-07-15


In [17]:
jhb_comb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14634 entries, 0 to 14633
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   rowid                 14634 non-null  int64         
 1   date                  14634 non-null  datetime64[ns]
 2   commodity             14634 non-null  object        
 3   container             14634 non-null  object        
 4   unit_mass             14634 non-null  int64         
 5   product_combination   14634 non-null  object        
 6   total_value_sold      14634 non-null  float64       
 7   total_qty_sold        14634 non-null  int64         
 8   total_kg_sold         14634 non-null  float64       
 9   average               14634 non-null  float64       
 10  highest_price         14634 non-null  float64       
 11  ave_per_kg            14634 non-null  float64       
 12  highest_price_per_kg  14634 non-null  float64       
dtypes: datetime64[ns

In [13]:
# Cleaning Johannesburg Dataframe
jhb_comb_df["date"]=pd.to_datetime(jhb_comb_df["date"])
jhb_comb_df=to_numeric(jhb_comb_df, "total_value_sold")
jhb_comb_df=to_numeric(jhb_comb_df, "average")
jhb_comb_df=to_numeric(jhb_comb_df, "highest_price")
jhb_comb_df=to_numeric(jhb_comb_df, "ave_per_kg")
jhb_comb_df=to_numeric(jhb_comb_df, "highest_price_per_kg")

C:\Users\27784\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [16]:
jhb_comb_df.head()

,rowid,date,commodity,container,unit_mass,product_combination,total_value_sold,total_qty_sold,total_kg_sold,average,highest_price,ave_per_kg,highest_price_per_kg
0,13803,2020-08-31,ONIONS,2KG POCKET,2,"SWEET,CL 1,L/M,*,NORTHERN CAPE",0.0,0,0.0,0.0,0.0,0.00,0.0
1,13804,2020-08-31,ONIONS,2KG POCKET,2,"WHITE,CL 1,L/M,*,NORTHERN CAPE",0.0,0,0.0,0.0,0.0,0.00,0.0
2,13805,2020-08-31,ONIONS,2KG POCKET,2,"WHITE,CL 1,M,*,NORTHERN CAPE",0.0,0,0.0,0.0,0.0,0.00,0.0
3,13806,2020-08-31,ONIONS,3KG POCKET,3,"BROWN,CL 1,L/M,*,NORTHERN CAPE",6138.0,361,1083.0,17.0,18.0,5.67,6.0
4,13807,2020-08-31,ONIONS,3KG POCKET,3,"BROWN,CL 1,L/M,*,WESTERN CAPE",0.0,0,0.0,0.0,0.0,0.00,0.0


In [27]:
jhb_con_df_cleaned=to_numeric(jhb_con_df_cleaned, "average_price_per_kg")

C:\Users\27784\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [28]:
woolworths_df=to_numeric(woolworths_df, "Price")
woolworths_df["Date"]=pd.to_datetime(woolworths_df["Date"])

pnp_df=to_numeric(pnp_df, "Price")
pnp_df["Date"]=pd.to_datetime(pnp_df["Date"])

shoprite_df=to_numeric(shoprite_df, "Price")
shoprite_df["Date"]=pd.to_datetime(shoprite_df["Date"])

## Creating an Engine to send the cleaned tables to RDS instace 

In [1]:
# Packages need
from sqlalchemy import create_engine

import urllib

params = urllib.parse.quote_plus()
engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)





## Adding the tables in the RDS instance

In [33]:
jhb_comm_df_cleaned.to_sql('Joburg_Fresh_produce_commodity_cleaned', con=engine)

In [46]:
jhb_con_df_cleaned.to_sql('Joburg_Fresh_produce_container_cleaned', con=engine)


In [ ]:
jhb_comb_df.to_sql('Joburg_Fresh_produce_combined_cleaned', con=engine)


In [ ]:
cpt_df.to_sql('Capetown_Fresh_produce_market_cleaned', con=engine)


In [ ]:
dbn_df.to_sql('Durban_Fresh_produce_market_cleaned', con=engine)


In [ ]:
woolworths_df.to_sql('Woolworths_Prices_cleaned', con=engine)


In [ ]:
pnp_df.to_sql('PickNPay_Prices_cleaned', con=engine)


In [ ]:
shoprite_df.to_sql('Shoprite_Prices_cleaned', con=engine)

In [ ]:
cur.commit()

## Interacting with the database in RDS to query the new created tables

In [34]:
engine.execute("SELECT * FROM Joburg_Fresh_produce_commodity_cleaned").fetchall()

[(0, 1, '20 August 2020', 'AMADUMBE', 2, 39870.0, 0.0, 0.0, 97.0, 0.0, 1940.0),
 (1, 2, '20 August 2020', 'APPLES', 91755, 22664221.0, 1205932.0, 13799.0, 261296.0, 157462.0, 3163863.0),
 (2, 3, '20 August 2020', 'ARTICHOKES', 1, 53100.0, 600.0, 4.0, 439.0, 3.0, 522.0),
 (3, 4, '20 August 2020', 'ASPARAGUS', 8, 258975.0, 34000.0, 50.0, 359.0, 250.0, 1795.0),
 (4, 5, '20 August 2020', 'ATCHARA', 207, 1351.2, 0.0, 0.0, 23.0, 0.0, 65.0),
 (5, 6, '20 August 2020', 'AVOCADOS', 14429, 11358074.4, 1058716.0, 9569.0, 124016.0, 54059.0, 747703.0),
 (6, 7, '20 August 2020', 'BABY BEET', 7, 1462.0, 0.0, 0.0, 31.0, 0.0, 63.0),
 (7, 8, '20 August 2020', 'BABY BRINJALS', 0, 1920.0, 1320.0, 9.0, 13.0, 27.0, 39.0),
 (8, 9, '20 August 2020', 'BABY BUTTERNUT', 6, 117371.0, 21752.0, 86.0, 425.0, 403.0, 1925.0),
 (9, 10, '20 August 2020', 'BABY CABBAGE', 197, 254455.0, 21824.0, 196.0, 2592.0, 783.0, 9422.0),
 (10, 11, '20 August 2020', 'BABY CARROTS', 301, 163250.0, 16210.0, 84.0, 946.0, 484.0, 4907.0),
 

In [49]:
engine.execute('SELECT * from Joburg_Fresh_produce_container_cleaned').fetchall()

[(0, 1, '20 August 2020', 'AMADUMBE', '20KG POCKET', 2, 0.0, 39870.0, 0.0, 0.0, 97.0, 0.0, 1940.0),
 (1, 2, '20 August 2020', 'APPLES', '10 X 1KG ECONO PACK CARTON', 17, 10.0, 41932.0, 100.0, 1.0, 547.0, 10.0, 5470.0),
 (2, 3, '20 August 2020', 'APPLES', '11KG JUMBLE CARTON', 343, 6.36, 218914.0, 1190.0, 17.0, 3170.0, 187.0, 34870.0),
 (3, 4, '20 August 2020', 'APPLES', '12 X 1KG ECONO PACK CARTON', 3233, 7.86, 1454572.0, 40738.0, 432.0, 17353.0, 5184.0, 208236.0),
 (4, 5, '20 August 2020', 'APPLES', '12.5KG M6 CARTON', 12, 0.0, 16903.0, 0.0, 0.0, 286.0, 0.0, 3575.0),
 (5, 6, '20 August 2020', 'APPLES', '12.5KG M6 CONTAINER', 0, 8.8, 7700.0, 7700.0, 70.0, 70.0, 875.0, 875.0),
 (6, 7, '20 August 2020', 'APPLES', '18.50KG CARTON', 34346, 8.25, 7339878.0, 411595.0, 2696.0, 53757.0, 49876.0, 994191.0),
 (7, 8, '20 August 2020', 'APPLES', '3 KG ECONO PACK BOX', 211, 0.0, 59208.0, 0.0, 0.0, 1490.0, 0.0, 6675.0),
 (8, 9, '20 August 2020', 'APPLES', '300KG BULK BIN', 129, 5.17, 1278425.0, 7755

In [48]:
#engine.execute('SELECT * from Joburg_Fresh_produce_combined_cleaned').fetchall()

In [43]:
#engine.execute('SELECT * from Capetown_Fresh_produce_market_cleaned').fetchall()

In [ ]:
engine.execute('SELECT * from Durban_Fresh_produce_market_cleaned').fetchall()

In [ ]:
engine.execute('SELECT * from Woolworths_Prices_cleaned').fetchall()

In [ ]:
engine.execute('SELECT * from PickNPay_Prices_cleaned').fetchall()

In [ ]:
engine.execute('SELECT * from Shoprite_Prices_cleaned').fetchall()

## Dropping a table

**We can drop any table if we want to recreate it, using the below code**

In [40]:
sql = 'Drop TABLE Joburg_Fresh_produce_container_cleaned;'
engine.execute(sql)
#cur.commit()

In [42]:
sql = 'Drop TABLE Joburg_Fresh_produce_combined_cleaned;'
engine.execute(sql)